
# Workflow Practice

In this notebook, you’ll practice connecting to a SQLite database, creating tables from CSV files using Pandas, and writing SQL queries to explore the data.

The dataset comes from the [Bike Store Sample Database](https://www.kaggle.com/datasets/dillonmyrick/bike-store-sample-database) by Dillon Myrick. It models a fictional bike retailer with multiple stores, products, customers, and staff. Each table connects to others using foreign keys such as `customer_id`, `store_id`, and `product_id`.

You’ll:
- Connect to a local SQLite database
- Create tables using `pandas.to_sql()`
- Write and test SQL queries using `pd.read_sql()`

All of your work will take place directly in this notebook. Each question prompt is written below as a Markdown cell, followed by an empty code cell for you to write your query.



## Step 1: Connect to the Database

Run the following cell to connect to (or create) a SQLite database called `bike_store.db`.  
If the file doesn’t exist yet, SQLite will automatically create it.


In [1]:
import sqlite3
import pandas as pd

In [2]:
connection = sqlite3.connect("bike_store.db")
connection


## Step 2: Create Tables from CSV Files

The `data/` folder contains one CSV file per table.  
Use `pandas.read_csv()` and `DataFrame.to_sql()` to load each file into your database.

You only need to do this once.  
After that, you’ll be able to run queries against your newly created tables.


In [4]:
# Example for one file
customers = pd.read_csv("data/customers.csv")
customers.to_sql("customers", connection, if_exists="replace", index=False)

1445

In [6]:
# Repeat for all other files in the data folder, or use a loop.
# List of CSV files and their table names
files = ["orders", "brands", "categories","order_items","products","staffs","stocks","stores"]
# orders = pd.read_csv("data/customers.csv")
# orders.to_sql("orders",connection, if_exists="replace", index=False)
for name in files:
    df = pd.read_csv(f"data/{name}.csv")
    df.to_sql(name, connection, if_exists="replace", index=False)


### Verify Your Tables

Run a query to make sure your tables were created successfully.

In [7]:

pd.read_sql("SELECT name FROM sqlite_master WHERE type='table';", connection)
#pd.read_sql()


,name
0,customers
1,orders
2,brands
3,categories
4,order_items
5,products
6,staffs
7,stocks
8,stores


## Step 3: Test a Simple Query

Before starting the exercises, confirm your connection and tables are working by previewing the first few rows of the `customers` table.

In [8]:

pd.read_sql("SELECT * FROM customers LIMIT 5;", connection)
# pd.read_sql("SELECT count(*) FROM customers LIMIT 5;", connection)  #1445
# customer_id	first_name	last_name	phone	email	street	city	state	zip_code


,customer_id,first_name,last_name,phone,email,street,city,state,zip_code
0,1,Debra,Burks,None,debra.burks@yahoo.com,9273 Thorne Ave.,Orchard Park,NY,14127
1,2,Kasha,Todd,None,kasha.todd@yahoo.com,910 Vine Street,Campbell,CA,95008
2,3,Tameka,Fisher,None,tameka.fisher@aol.com,769C Honey Creek St.,Redondo Beach,CA,90278
3,4,Daryl,Spence,None,daryl.spence@aol.com,988 Pearl Lane,Uniondale,NY,11553
4,5,Charolette,Rice,(916) 381-6003,charolette.rice@msn.com,107 River Dr.,Sacramento,CA,95820


### Q1. List all customers and their cities.

Return the first name, last name, and city of each customer. Sort alphabetically by last name and then by first name.

In [9]:
# Your query here
query1 = """
    SELECT last_name,first_name,city FROM customers 
    ORDER BY last_name,first_name;
    """
pd.read_sql(query1, connection)


,last_name,first_name,city
0,Acevedo,Ester,San Lorenzo
1,Acevedo,Jamika,Ozone Park
2,Acevedo,Penny,Ballston Spa
3,Acosta,Bettyann,Lancaster
4,Acosta,Shery,Saratoga Springs
...,...,...,...
1440,Young,Edda,North Tonawanda
1441,Young,Jasmin,Helotes
1442,Zamora,Alexandria,Schenectady
1443,Zamora,Jayme,Springfield Gardens


### Q2. Show all products and their prices.

Display each product name along with its list price. Sort by price in descending order.

In [10]:
# Your query here
query2 = """
    SELECT product_name, "$"||list_price as list_price
    FROM products 
    ORDER BY list_price desc;
    """
pd.read_sql(query2, connection)
#pd.read_sql("SELECT * FROM products LIMIT 5;", connection)
#pd.read_sql("SELECT count(*) FROM products LIMIT 5;", connection) #321
# product_id	product_name	brand_id	category_id	model_year	list_price

,product_name,list_price
0,Surly Wednesday Frameset - 2016,$999.99
1,Surly Big Dummy Frameset - 2017,$999.99
2,Trek X-Caliber 8 - 2017,$999.99
3,Surly Ice Cream Truck Frameset - 2017,$999.99
4,Trek X-Caliber 8 - 2018,$999.99
...,...,...
316,Trek Domane SLR 9 Disc - 2018,$11999.99
317,Electra Amsterdam Royal 8i Ladies - 2018,$1199.99
318,Electra Amsterdam Fashion 7i Ladies' - 2017,$1099.99
319,Trek Domane ALR 3 - 2018,$1099.99


### Q3. Find all customers from California.

Return first name, last name, city, and state for all customers whose state is 'CA'. Sort alphabetically by last name.

In [11]:
# Your query here
query3 = """
    SELECT first_name,last_name,city,state FROM customers 
    WHERE state = "CA"
    ORDER BY last_name,first_name;
    """
pd.read_sql(query3, connection)

,first_name,last_name,city,state
0,Ester,Acevedo,San Lorenzo,CA
1,Jamaal,Albert,Torrance,CA
2,Sindy,Anderson,Pomona,CA
3,Twana,Arnold,Anaheim,CA
4,Selene,Austin,Duarte,CA
...,...,...,...,...
279,Darren,Witt,Coachella,CA
280,Lucy,Woods,Palos Verdes Peninsula,CA
281,Joel,Wynn,San Diego,CA
282,Yvone,Yates,San Pablo,CA


### Q4. Count how many products are in each category.

Return the category name and the number of products in that category. Sort from the highest count to the lowest.

In [12]:
# Your query here
query4 = """
    SELECT products.category_id,categories.category_name, COUNT(products.category_id) AS no_of_products 
    FROM products JOIN categories ON products.category_id = categories.category_id
    GROUP BY products.category_id,categories.category_name
    ORDER BY no_of_products DESC
    ;
    """
pd.read_sql(query4, connection)
#pd.read_sql("SELECT * FROM products LIMIT 5;", connection)
#pd.read_sql("SELECT * FROM categories LIMIT 5;", connection) # 7 categories

,category_id,category_name,no_of_products
0,3,Cruisers Bicycles,78
1,6,Mountain Bikes,60
2,7,Road Bikes,60
3,1,Children Bicycles,59
4,2,Comfort Bicycles,30
5,5,Electric Bikes,24
6,4,Cyclocross Bicycles,10


### Q5. Find all orders placed in 2018.

List the order ID, order date, and customer ID for orders made during the year 2018. Sort by order date.

In [13]:
# Your query here
query5 = """
    SELECT order_id, order_date,  customer_id
    FROM orders
    WHERE strftime('%Y', order_date)= '2018'
    ORDER BY order_date
    ;
    """
pd.read_sql(query5, connection)
#pd.read_sql("SELECT * FROM orders;", connection) #1615
# order_id	customer_id	order_status	order_date	required_date	shipped_date	store_id	staff_id

,order_id,order_date,customer_id
0,1324,2018-01-01,862
1,1325,2018-01-01,68
2,1326,2018-01-01,567
3,1327,2018-01-02,1026
4,1328,2018-01-02,1083
...,...,...,...
287,1611,2018-09-06,6
288,1612,2018-10-21,3
289,1613,2018-11-18,1
290,1614,2018-11-28,135


### Q6. Show each order with its total number of items.

Join the `orders` and `order_items` tables. Group by order ID and return the number of items per order.

In [87]:
# Your query here
query6 = """
    SELECT oi.order_id, COUNT(oi.item_id) AS no_of_items_per_order
    FROM orders o JOIN order_items oi
    ON o.order_id = oi.order_id
    GROUP BY oi.order_id;
    """
pd.read_sql(query6, connection)
#pd.read_sql("SELECT * FROM order_items;", connection)
# order_id	item_id	product_id	quantity	list_price	discount

,order_id,no_of_items_per_order
0,1,5
1,2,2
2,3,2
3,4,1
4,5,3
...,...,...
1610,1611,3
1611,1612,5
1612,1613,2
1613,1614,3


### Q7. List total revenue per store.

Revenue = quantity * list_price * (1 - discount). Join `orders`, `order_items`, and `stores`, group by store name, and return total revenue.

In [19]:
# Your query here
query7 = """
    SELECT s.store_name, "$"||ROUND(FORMAT(SUM(oi.quantity*oi.list_price * (1-oi.discount)), 'C', 'en-US'),2) AS total_revenue
    FROM orders o JOIN order_items oi
    ON o.order_id = oi.order_id
    JOIN stores s ON o.store_id = s.store_id
    GROUP BY s.store_name
    ;
    """
pd.read_sql(query7, connection)
# GROUP BY s.store_name
#pd.read_sql("SELECT * FROM stores;", connection)
#store_id	store_name	phone	email	street	city	state	zip_code

,store_name,total_revenue
0,Baldwin Bikes,$5215751.28
1,Rowlett Bikes,$867542.24
2,Santa Cruz Bikes,$1605823.04


In [18]:
pd.read_sql("SELECT * FROM stores;", connection)

,store_id,store_name,phone,email,street,city,state,zip_code
0,1,Santa Cruz Bikes,(831) 476-4321,santacruz@bikes.shop,3700 Portola Drive,Santa Cruz,CA,95060
1,2,Baldwin Bikes,(516) 379-8888,baldwin@bikes.shop,4200 Chestnut Lane,Baldwin,NY,11432
2,3,Rowlett Bikes,(972) 530-5555,rowlett@bikes.shop,8000 Fairway Avenue,Rowlett,TX,75088


### Q8. Find the top 5 customers who spent the most overall.

Join `customers`, `orders`, and `order_items`. Sum the total spending per customer and return the top five spenders.

In [88]:
# Your query here
query8 =  """
    SELECT c.customer_id,c.first_name,c.last_name, round(SUM((oi.quantity*oi.list_price* (1-oi.discount))),2) AS total_spending_per_customer
    FROM orders o JOIN order_items oi
    ON o.order_id = oi.order_id
    JOIN customers c ON o.customer_id = c.customer_id
    GROUP BY o.customer_id,c.first_name,c.last_name
    ORDER BY total_spending_per_customer DESC
    LIMIT 5;
    """
pd.read_sql(query8,connection)

,customer_id,first_name,last_name,total_spending_per_customer
0,94,Sharyn,Hopkins,34807.94
1,10,Pamelia,Newman,33634.26
2,75,Abby,Gamble,32803.01
3,6,Lyndsey,Bean,32675.07
4,16,Emmitt,Sanchez,31925.89


In [ ]:
pd.read_sql("SELECT * FROM order_items;", connection)
#order_id	item_id	product_id	quantity	list_price	discount
pd.read_sql("SELECT * FROM orders;", connection)
# order_id	customer_id	order_status	order_date	required_date	shipped_date	store_id	staff_id


### Q9. Show the best-selling product in each category.

Join `products`, `order_items`, and `categories`. For each category, identify the product with the highest total quantity sold.

In [89]:
query9 = """
   WITH prod_oi_cat AS
    (SELECT cat.category_id,cat.category_name,p.product_id, p.product_name, SUM(oi.quantity) AS total_product_qty_sold
    FROM order_items oi JOIN products p
    ON  oi.product_id = p.product_id
    JOIN categories cat ON p.category_id = cat.category_id
    GROUP BY p.product_id
    ORDER BY total_product_qty_sold DESC)
    
    SELECT category_id, category_name, product_id, product_name, total_product_qty_sold
    FROM (
        SELECT cat.category_id,cat.category_name,cat.product_id, cat.product_name, cat.total_product_qty_sold, 
        ROW_NUMBER() OVER ( PARTITION BY cat.category_id ORDER BY total_product_qty_sold desc) AS category_rank
        FROM prod_oi_cat cat 
    ) 
    WHERE category_rank= 1;
    """
pd.read_sql(query9, connection)

,category_id,category_name,product_id,product_name,total_product_qty_sold
0,1,Children Bicycles,23,Electra Girl's Hawaii 1 (20-inch) - 2015/2016,154
1,2,Comfort Bicycles,25,Electra Townie Original 7D - 2015/2016,148
2,3,Cruisers Bicycles,13,Electra Cruiser 1 (24-Inch) - 2016,157
3,4,Cyclocross Bicycles,11,Surly Straggler 650b - 2016,151
4,5,Electric Bikes,9,Trek Conduit+ - 2016,145
5,6,Mountain Bikes,6,Surly Ice Cream Truck Frameset - 2016,167
6,7,Road Bikes,56,Trek Domane SLR 6 Disc - 2017,43


In [ ]:
pd.read_sql("SELECT * FROM products;", connection)
#product_id	product_name	brand_id	category_id	model_year	list_price
pd.read_sql("SELECT * FROM categories;", connection)
# category_id	category_name
pd.read_sql("SELECT * FROM order_items;", connection)
#order_id	item_id	product_id	quantity	list_price	discount
pd.read_sql("SELECT * FROM order_items GROUP BY order_id;", connection)


### Q10. Identify the employees (staff) who processed the most orders.

Join `staffs` and `orders`. Count the number of orders handled by each staff member and return the results sorted by highest total.

In [90]:
# Your query here
pd.read_sql("SELECT * FROM staffs;", connection)
#staff_id	first_name	last_name	email	phone	active	store_id	manager_id
pd.read_sql("SELECT * FROM orders;", connection)
#order_id	customer_id	order_status	order_date	required_date	shipped_date	store_id	staff_id
query10 =  """
    SELECT st.staff_id,st.first_name,st.last_name, COUNT(o.order_id) AS no_of_orders_handled_staff
    FROM orders o JOIN staffs st
    ON o.staff_id = st.staff_id
    GROUP BY st.staff_id
    ORDER BY no_of_orders_handled_staff DESC
   
    ;
    """
pd.read_sql(query10, connection)

,staff_id,first_name,last_name,no_of_orders_handled_staff
0,6,Marcelene,Boyer,553
1,7,Venita,Daniel,540
2,3,Genna,Serrano,184
3,2,Mireya,Copeland,164
4,8,Kali,Vargas,88
5,9,Layla,Terrell,86
